# Credentials and initialize conneciton

Follow https://developers.google.com/earth-engine/reference/Quickstart#before-you-begin to get a secret key

In [2]:
import json

KEY = 'my-secret-key.json'
with open(KEY, 'r') as f:
    data = json.load(f)
    SERVICE_ACCOUNT = data['client_email']
    PROJECT = data['project_id']

In [3]:
import ee

ee_creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
ee.Initialize(ee_creds)

# NDVI around coordinate

In [61]:
import geemap

def mask_s2_clouds(image):
    scl = image.select('SCL')
    # Mask classes: clouds (3,8,9,10), cloud shadow (3), maybe snow (11)
    cloud_classes = [0,1, 2, 3, 8, 9, 10]
    mask = scl.remap(cloud_classes, [0]*len(cloud_classes), 1).eq(1)
    return image.updateMask(mask)

def add_ndvi_and_index(image, index):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    index_band = ee.Image.constant(index).rename('index').toFloat()
    return image.addBands([ndvi, index_band])

# Define 1km bbox near Nairobi
geom = ee.Geometry.BBox(36.9, 0.4, 37.1, 0.6)

# Filter collection and mask clouds
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geom)
    .filterDate('2025-06-01', '2025-06-10')
    .map(mask_s2_clouds)
    .sort('system:time_start')  # oldest first
)

images_list = collection.toList(collection.size())
dates = []
indexed_images = []

# Build new images with NDVI + index bands
for i in range(images_list.size().getInfo()):
    img = ee.Image(images_list.get(i))
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    dates.append(date_str)
    indexed_img = add_ndvi_and_index(img, i)
    indexed_images.append(indexed_img)

indexed_collection = ee.ImageCollection(indexed_images)

# qualityMosaic by 'index' to get newest pixel per location (index highest = newest)
filled = indexed_collection.qualityMosaic('index').clip(geom)
ndvi = filled.select('NDVI')
rgb = filled.select(['B4', 'B3', 'B2']).divide(10000)  # Scale RGB bands to [0, 0.3]
index = filled.select('index')


# Visualization parameters
ndvi_vis = {
    'min': 0,
    'max': 1.0,
    'palette': ['red', 'yellow' 'green']
}

index_vis = {
    'min': 0,
    'max': len(dates) - 1,
    'palette': ['black', 'white']  # white=oldest, black=newest
}

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
}

# Create the map
Map = geemap.Map(center=[0.5, 37], zoom=15)
Map.addLayer(ndvi, ndvi_vis, 'Cloud-Filled NDVI')
Map.addLayer(index, index_vis, 'Image Recency Index (black=newest)')
Map.addLayer(rgb, rgb_vis, 'RGB Composite')

# Print index-to-date mapping
print("Index to Date mapping:")
for idx, d in enumerate(dates):
    print(f"Index {idx}: {d}")

Map


Index to Date mapping:
Index 0: 2025-06-01
Index 1: 2025-06-06
Index 2: 2025-06-08


Map(center=[0.5, 37], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

# Dynamic World around coordinate

In [50]:
import geemap
import ee

# Define bbox near Nairobi
geom = ee.Geometry.BBox(36.9, 0.4, 37.1, 0.6)

# Load Dynamic World image collection (most recent first)
collection = (
    ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
    .filterBounds(geom)
    .filterDate('2025-06-01', '2025-06-10')
    .sort('system:time_start', False)
)

# Extract only the 'grass' probability band and clip to geometry
grass_collection = collection.map(lambda img: img.select('grass').clip(geom))

# Convert to a list
grass_list = grass_collection.toList(grass_collection.size())

# Get first image
first = ee.Image(grass_list.get(0))

# Define the iterative temporal filling function
def fill_temporally(img, prev):
    img = ee.Image(img)
    prev = ee.Image(prev)
    return prev.blend(img)


# Iterate over the rest of the list
rest = grass_list.slice(1)
filled_grass = ee.Image(rest.iterate(fill_temporally, first))

# Visualization parameters
grass_vis = {
    'min': 0,
    'max': 1,
    'palette': ['ffffff', '88b053']  # white to green
}

# Display
Map = geemap.Map(center=[0.5, 37], zoom=14)
Map.addLayer(filled_grass, grass_vis, 'Grass Probability (Temporally Filled)')
Map

# Print the date of the most recent image
most_recent = ee.Image(collection.first())
print("Most recent image date:", ee.Date(most_recent.get('system:time_start')).format('YYYY-MM-dd').getInfo())

Map


Most recent image date: 2025-06-08


Map(center=[0.5, 37], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…